# Heat Equation

The heat equation is a typical example of parabolic equations. We search for the temperature $u : \Omega \times[0,T] \rightarrow R$ satisfying 

$$
\frac{\partial u}{\partial t}(x,t) - \Delta u(x,t) = f(x,t) \qquad \forall \, x \in \Omega, \forall t \in (0,T),
$$

where $f$ is a given heat source density. Here, $\Omega \subset R^d$ is the spatial domain, and $[0,T]$ is the time interval. In addition, we need initial conditions 

$$
u(x,0) = u_0(x) \qquad \forall \, x \in \Omega,
$$

and boundary conditions at the spatial boundary. For example Dirichlet boundary conditions read as

$$
u(x,t) = u_D(x,t) \qquad \forall \, x \in \partial \Omega, \; \forall  t \in(0,T]
$$

## Variational formulation in space

Similar as for elliptic equations we pass over to the weak formulations. We do this only for the spatial variables $x$, for all $t \in (0,T)$. The test functions $v$ depend only on $x$. We multiply by $v$, integration over $\Omega$, and perform integration by parts to obtain:

$$
\int_\Omega \frac{\partial u}{\partial t}(x,t) v(x) \, dx + \int \nabla u(x,t) \nabla v(x) \, dx = \int_\Omega f(x,t) v(x) \, dx \qquad \forall \, v \in H_0^1(\Omega) \; \forall t \, \in (0,T)
$$

The initial condition in $L_2$ is set for $t = 0$:
$$
u(\cdot, 0) = u_0 \qquad \text{in} \; L_2(\Omega)
$$

We consider the unknown function $u$ as a function with values in the Hilbert space $H^1$, e.g. $u : [0,T] \rightarrow H^1(\Omega)$. This allows to express the equation as Hilbert-space valued ordinary differential equation:
$$
\frac{d u }{dt }(t) + A u(t) = f(t)
$$
... some more definitions to come ...

## Galerkin method in space

As a next step we apply the finite element method in space. Let $V_h$ be a finite element sub-space of $H^1(\Omega)$, with the basis functions $\{ p_1(x), \ldots p_N(x) \}$. We approximate the time-dependent function by

$$
u_h : [0,T] \rightarrow V_h,
$$

which can we expressed by means of the basis as

$$
u_h(t) = \sum_{i=1}^N u_i(t) p_i(x)
$$

Inserting this expansion into the variational formulation, and using spatial test-functions $v = p_j$ we obtain the semi-discrete equation:


$$
\int_\Omega \frac{\partial}{\partial t} \Big\{ \sum_{i=1}^N u_i(t) p_i(x) \Big\} p_j(x) dx + 
\int_\Omega \nabla \Big\{ \sum_{i=1}^N u_i(t) p_i(x) \Big\} \; \nabla p_j(x) = \int_\Omega f(x,t) p_j(x) \, dx
\qquad \forall \, j = 1 \ldots N, \forall \, t \in (0,T)
$$


The initial condition for $u_h$ is obtained by some kind of projection, typically we want the $L_2$ projection

$$
\int u_h(x,0) p_j(x) \, dx = \int u_0(x) p_j(x) \, dx \qquad \forall \, j \in \{ 1, \ldots N \}
$$





By means of the **mass matrix** $M \in R^{N \times N}$ 
$$
M = \left\{ \int_\Omega p_i p_j \, dx \right\}_{i,j = 1, \ldots N}
$$
and **stiffness matrix** $A \in R^{N \times N}$ 
$$
A = \left\{ \int_\Omega \nabla p_i \nabla p_j \, dx \right\}_{i,j = 1, \ldots N}
$$
we obtain the ordinary differential equation (ODE)

$$
M \dot u(t) + A u(t) = f(t) \qquad \forall \, t 
$$

including the initial condition 

$$
u(0) = u_0
$$

for the unknown coefficient function $u(\cdot) = (u_1(\cdot), \ldots u_N(\cdot)) \in C^1([0,T], R^N)$.

# Implicit Euler time-stepping

We approximate the function $u$ at time-steps $t_j = j \tau$ by $u^j$, with the time-step $\tau$ and $j \in \{0, \ldots m\}$. By replacing the time-derivative $\dot u$ by the backward difference quotient we obtain

$$
M \frac{u^{j}-u^{j-1}}{\tau} + A u^j = f^j
$$

The initial value $u^0 := u_0$ is given by the initial condition. If the old time-step solution $u^{j-1}$ is known, we can compute the next step by the linear system of equations

$$
(M + \tau A) u^j = \tau f + M u^{j-1}.
$$

Often the new step is computed in incremental form:

\begin{eqnarray*}
(M + \tau A) w^j &=& f - A u^{j-1} \\
u^j & = & u^{j-1} + \tau w^j
\end{eqnarray*}

In [9]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import unit_square
from time import sleep

mesh = Mesh(unit_square.GenerateMesh(maxh=0.05))

setting up matrices:

In [10]:
tau = 0.1
tend = 1
u0 = x*y*(x-1)*(y-1)

In [11]:
t = 0
exact = exp(-t)*x*y*(x-1)*(y-1)
source = - exp(-t)*x*y*(x-1)*(y-1) - exact.Diff(x).Diff(x) - exact.Diff(y).Diff(y)

fes = H1(mesh, order=1, dirichlet='left|right|bottom|top')
u,v = fes.TnT()
mform = u*v*dx
aform = grad(u)*grad(v)*dx
fform = source * v * dx

m = BilinearForm(mform).Assemble()
a = BilinearForm(aform).Assemble()
mstar = BilinearForm(mform+tau*aform).Assemble()
mstarinv = mstar.mat.Inverse(fes.FreeDofs())
f = LinearForm(fform).Assemble()


In [12]:
gfu = GridFunction(fes)
gfu.Set(u0)

# Draw(exact, mesh)
scene = Draw (gfu, deformation=True)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

In [13]:
errors = []
for j in range(int(tend/tau)):
    print('j=', j, 't=', t)
    res = f.vec - a.mat * gfu.vec
    w = mstarinv * res
    gfu.vec.data += tau*w
    err = sqrt(Integrate((gfu-exact)**2, mesh))
    print(gfu.vec[100:103])
    errors.append(err)
    print(f.vec[:3])
    t = tau*(j+1)
    exact = exp(-t)*x*y*(x-1)*(y-1)
    source = - exp(-t)*x*y*(x-1)*(y-1) - exact.Diff(x).Diff(x) - exact.Diff(y).Diff(y)
    fform = source * v * dx
    f = LinearForm(fform).Assemble()
    scene.Redraw()
    sleep(0.2)

j= 0 t= 0
 0.00422845
 0.00584934
 0.00712884


 2.03125e-05
 4.41069e-05
 2.03125e-05


j= 1 t= 0.1
 0.00390084
 0.00540006
 0.00658428


 1.83795e-05
 3.99096e-05
 1.83795e-05


j= 2 t= 0.2
 0.00355434
 0.00492257
 0.00600367


 1.66305e-05
 3.61117e-05
 1.66305e-05


j= 3 t= 0.30000000000000004
 0.00322536
 0.00446848
 0.00545075


 1.50479e-05
 3.26752e-05
 1.50479e-05


j= 4 t= 0.4
 0.00292262
 0.00405035
 0.00494135


 1.36159e-05
 2.95657e-05
 1.36159e-05


j= 5 t= 0.5
 0.002647
 0.00366958
 0.00447739


 1.23202e-05
 2.67522e-05
 1.23202e-05


j= 6 t= 0.6000000000000001
 0.00239704
 0.00332424
 0.00405656


 1.11477e-05
 2.42064e-05
 1.11477e-05


j= 7 t= 0.7000000000000001
 0.00217068
 0.00301149
 0.00367544


 1.00869e-05
 2.19028e-05
 1.00869e-05


j= 8 t= 0.8
 0.0019658
 0.00272841
 0.00333047


 9.12699e-06
 1.98185e-05
 9.12699e-06


j= 9 t= 0.9
 0.00178039
 0.00247224
 0.00301829


 8.25845e-06
 1.79325e-05
 8.25845e-06




In [16]:
errors

[0.001148599785951597,
 0.0003597200845947036,
 0.00010116570031072275,
 4.415862048380539e-05,
 5.289910843353487e-05,
 5.9402384893122526e-05,
 6.216108588046966e-05,
 6.351649467995372e-05,
 6.44379989595385e-05,
 6.524882252443768e-05]

In [17]:
scene = Draw(gfu, deformation=True)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

In [18]:
mesh_size = [0.5, 0.25, 0.125, 0.0625, 0.03125]#, 0.015625]
dofs = []
errors = []

for h in mesh_size:
    t = 0
    exact = exp(-t)*x*y*(x-1)*(y-1)
    source = - exp(-t)*x*y*(x-1)*(y-1) - exact.Diff(x).Diff(x) - exact.Diff(y).Diff(y)
    
    mesh = Mesh(unit_square.GenerateMesh(maxh=h))
    fes = H1(mesh, order=2, dirichlet='left|right|bottom|top')
    dofs.append(fes.ndof)
    u,v = fes.TnT()
    mform = u*v*dx
    aform = grad(u)*grad(v)*dx
    fform = source * v * dx

    m = BilinearForm(mform).Assemble()
    a = BilinearForm(aform).Assemble()
    mstar = BilinearForm(mform+tau*aform).Assemble()
    mstarinv = mstar.mat.Inverse(fes.FreeDofs())
    f = LinearForm(fform).Assemble()

    gfu = GridFunction(fes)
    gfu.Set(u0)
#     exact_scene = Draw(exact, mesh)
#     scene = Draw (gfu, deformation=True)
    errs = []
    for j in range(int(tend/tau)):
        res = f.vec - a.mat * gfu.vec
        w = mstarinv * res
        gfu.vec.data += tau*w
        err = sqrt(Integrate((gfu-exact)*(gfu-exact), mesh))
        errs.append(err)
#         t.Set(tau*(j+1))
        t = tau*(j+1)
        exact = exp(-t)*x*y*(x-1)*(y-1)
        source = - exp(-t)*x*y*(x-1)*(y-1) - exact.Diff(x).Diff(x) - exact.Diff(y).Diff(y)
        fform = source * v * dx
        f = LinearForm(fform).Assemble()
        
#         exact_scene.Redraw()
#         scene.Redraw()
        sleep(0.2)    
    errors.append(min(errs))

In [19]:
errors

[0.0008445798090612784,
 0.00010768814114864109,
 2.8382586415541855e-05,
 2.2309337104405597e-05,
 2.2192184959082396e-05]

In [20]:
import numpy as np

for i in range(10):
    print('t=', tau*(i+1))
    for j in range(len(errors)-1):
        conv_rate = np.log(errors[j][i+1]/errors[j][i])/np.log(mesh_size[j+1]/mesh_size[j])
        print("Convergence rate:", conv_rate, '\n')


t= 0.1


TypeError: 'float' object is not subscriptable

In [ ]:
import numpy as np
for i in range(len(mesh_size)-1):
    conv_rate = np.log(errors[i+1]/errors[i])/np.log(mesh_size[i+1]/mesh_size[i])
    print("Convergence rate:", conv_rate)
